In [2]:
# making my own neural network

import numpy as np

class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # set number of nodes in each input, hidden, output layer
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        
        # set learning rate
        self.learning_rate = learning_rate
        
        # initialize weights
        self.weights_input_to_hidden = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                                        (self.hidden_nodes, self.input_nodes))
        
        self.weights_hidden_to_output = np.random.normal(0.0, self.output_nodes**-0.5, 
                                                         (self.output_nodes, self.hidden_nodes))
        
        # initialize bias
        self.bias_hidden = np.random.normal(0.0, self.hidden_nodes**-0.5, (self.hidden_nodes, 1))
        self.bias_output = np.random.normal(0.0, self.output_nodes**-0.5, (self.output_nodes, 1))

        # set activation function
        self.activation_function = lambda x : 1 / (1 + np.exp(-x))
        
    def train(self, features, targets):
        # number of records
        n_records = features.shape[0]
        
        # initialize delta weights
        delta_weights_i_h = np.zeros(self.weights_input_to_hidden.shape)
        delta_weights_h_o = np.zeros(self.weights_hidden_to_output.shape)
        
        # initialize delta bias
        delta_bias_h = np.zeros(self.bias_hidden.shape)
        delta_bias_o = np.zeros(self.bias_output.shape)
        
        for X, y in zip(features, targets):
            # feedforward
            final_outputs, hidden_outputs = self.feedforward(X)
            
            # backpropagation
            delta_weights_i_h, delta_weights_h_o, delta_bias_h, delta_bias_o = self.backpropagation(X, y, final_outputs, hidden_outputs,
                                                                                                    delta_weights_i_h, delta_weights_h_o,
                                                                                                    delta_bias_h, delta_bias_o)
            
            # update weights
            self.update_weights(delta_weights_i_h, delta_weights_h_o, delta_bias_h, delta_bias_o, n_records)

    def feedforward(self, X):
        # hidden layer
        hidden_inputs = np.dot(self.weights_input_to_hidden, X) + self.bias_hidden
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # output layer
        final_inputs = np.dot(self.weights_hidden_to_output, hidden_outputs) + self.bias_output
        final_outputs = final_inputs
        
        return final_outputs
    
    def MSE(self, y, Y):
        return np.mean((y-Y)**2)
    
    def backpropagation(self, X, y, final_outputs, hidden_outputs, delta_weights_i_h, delta_weights_h_o, delta_bias_h, delta_bias_o):
        # output error
        error = y - final_outputs
        
        # hidden error
        hidden_error = np.dot(self.weights_hidden_to_output.T, error)
        
        # backpropagated error terms
        output_error_term = error
        hidden_error_term = hidden_error * hidden_outputs * (1 - hidden_outputs)
        
        # weight step (input to hidden)
        delta_weights_i_h += hidden_error_term * X[:, None]
        
        # weight step (hidden to output)
        delta_weights_h_o += output_error_term * hidden_outputs[:, None]
        
        # bias step (hidden)
        delta_bias_h += hidden_error_term
        
        # bias step (output)
        delta_bias_o += output_error_term
        
        return delta_weights_i_h, delta_weights_h_o, delta_bias_h, delta_bias_o
    
    def update_weights(self, delta_weights_i_h, delta_weights_h_o, delta_bias_h, delta_bias_o, n_records):
        # update hidden-to-output weights with gradient descent step
        self.weights_hidden_to_output += self.learning_rate * delta_weights_h_o / n_records
        
        # update input-to-hidden weights with gradient descent step
        self.weights_input_to_hidden += self.learning_rate * delta_weights_i_h / n_records
        
        # update hidden bias with gradient descent step
        self.bias_hidden += self.learning_rate * delta_bias_h / n_records
        
        # update output bias with gradient descent step
        self.bias_output += self.learning_rate * delta_bias_o / n_records

    def run(self, features):
        # run a forward pass through the network
        hidden_inputs = np.dot(self.weights_input_to_hidden, features) + self.bias_hidden
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = np.dot(self.weights_hidden_to_output, hidden_outputs) + self.bias_output
        final_outputs = final_inputs
        
        return final_outputs

In [12]:
import pandas as pd

column_features = ['Forehead_width','Cheeks_width','Jawline_length','Face_length','Face_Type']
df = pd.read_csv('faces_data.csv', names=column_features)
target = {'square': 0, 'triangle': 1, 'diamond': 2, 'oblong': 3, 'oval': 4, 'heart': 5}
# df['Face_Type'] = df['Face_Type'].map(targets)
df['Face_Type'] = df['Face_Type'].apply(lambda x: target[x])

df.head()

,Forehead_width,Cheeks_width,Jawline_length,Face_length,Face_Type
0,13.734412,12.367397,17.727732,18.044504,0
1,20.422939,11.449610,15.774867,19.750843,0
2,13.169055,11.898801,13.916852,22.163556,0
3,12.270533,11.923147,16.648587,16.816854,0
4,15.848748,14.381366,16.513414,15.231728,0


In [17]:
# X and Y

features = df.drop('Face_Type', axis=1)
targets = df['Face_Type']

features.head()

,Forehead_width,Cheeks_width,Jawline_length,Face_length
0,13.734412,12.367397,17.727732,18.044504
1,20.422939,11.449610,15.774867,19.750843
2,13.169055,11.898801,13.916852,22.163556
3,12.270533,11.923147,16.648587,16.816854
4,15.848748,14.381366,16.513414,15.231728


In [18]:
# split data into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

n_input = 4
n_hidden = 3
n_output = 1
learning_rate = 0.1

nn = NeuralNetwork(n_input, n_hidden, n_output, learning_rate)

nn.train(X_train, y_train)

predictions = nn.run(X_test)

print(predictions)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U14')) -> None